# Time-Spacial Exploration

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make charts from `repteis` database with focus on time and space exploration. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.m
    

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
# alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig', low_memory=False)

<br>

<font size=5>**Color palette per Order**</font>

<!-- <img src="./src/paleta_cores.jpeg" width='500px'> -->

<!-- Cores: 

- verde_escuro: #284021
- verde_claro: #88BF11
- amarelo: #D9CB0B
- laranja: #D99311
- laranja_escuro: #BF4417
- marrom-_laro: #BF8D7A -->

In [3]:
# importing customized color palettes
from src.MNViz_colors import *

In [4]:
# p.s.: Caudata is an error and should be removed
# cores_ordem = {
#     'Squamata': '#BF4417',
#     'Testudines': '#D9CB0B', 
#     'Crocodylia': '#284021'
# }

ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

<br>

## Graph: Exploring time-spacial evolution of the database

In [5]:
# corrects a typo (Améica do Sul)
NewTable['continente'] = NewTable['continente'].apply(lambda x: 'América do Sul' if x=='Améica do Sul' else x)

In [6]:
# looking good...
NewTable['continente'].value_counts()

América do Sul      21927
América Central       135
América do Norte      103
Ásia                   54
#N/D                   25
África                 13
Europa                  9
Name: continente, dtype: int64

### Chart 1: Evolution per time and continent

In [7]:
# agrupando por Ano e Continente
teste = NewTable.groupby(['ano_coleta','continente', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [8]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# database
db = teste

# auxiliar variables for encoding fields
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
y_labels = db['continente'].unique()
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 100))

g1 = alt.Chart(db, title='Temporal evolution per continent', height=200, width=1400).mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year', 
             scale= alt.Scale(domain= x_labels)),
    y= alt.Y('continente', title='Continent', 
             scale= alt.Scale(domain= y_labels),
             sort=alt.EncodingSortField('counts', op="count", order='descending')), 
    size=alt.Size('counts', title='Counts',
                  legend= alt.Legend(orient= 'bottom', direction='horizontal'),
                  scale= alt.Scale(domain= counts, range=[20,120])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
#     color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),  # old palette per order
    color= alt.Color('familia',title= 'Family', 
                    legend= alt.Legend(columns=8, symbolLimit= 50, orient= 'bottom', direction='horizontal'),
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['continente','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_continent-wFamilies.html')
# g1

<br>

### Chart 2: Evolution per time and country

In [9]:
# agrupando por Ano e Continente
teste1 = NewTable.groupby(['ano_coleta','pais', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

In [10]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# filtering database (removing Brazil)
db = teste1[teste1['pais'] != 'Brasil']

# auxiliar variables for encoding fields
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
y_labels = db.groupby('pais').sum().reset_index().sort_values('counts')['pais'][::-1].unique()
temp = db['counts'].unique()
counts = list(range(temp.min(), temp.max()))

g1 = alt.Chart(db, title='Collected specimens per year in other countries',
              width=800).mark_circle().encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year', 
             scale= alt.Scale(domain= x_labels)),
    y= alt.Y('pais', title='Country', 
             scale= alt.Scale(domain= y_labels),
             sort=alt.EncodingSortField('counts', op="sum", order='descending')), 
    size=alt.Size('counts', title= 'Counts',
                  legend= alt.Legend(orient= 'right', direction='horizontal', tickCount=4),
                  scale= alt.Scale(domain= counts, range=[10,50])), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
#     color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),  # old palette per order
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['pais','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_country.html')
# g1

<br>

### Chart 3: Evolution per time and brazilian states

In [11]:
# agrupando por Ano e Continente
teste2 = NewTable[NewTable['pais'] == 'Brasil']
teste2 = teste2.groupby(['ano_coleta','estado_ou_provincia', 'familia']).count()['class'].reset_index().rename(columns={
    'class':'counts'
})

#### creating column with brazilian regions

In [13]:
regioes = {
    'Rio de Janeiro':'SE',
    'São Paulo':'SE',
    'Espírito Santo': 'SE',
    'Pernambuco':'NE',
    'Santa Catarina':'S',
    'Amazonas':'N',
    'Goiás':'CO',
    'Roraima':'N',
    'Pará':'N',
    'Mato Grosso':'CO',
    'Acre': 'N',
    'Bahia': 'NE',
    'Minas Gerais': 'SE',
    'Mato Grosso do Sul': 'CO',
    'Paraná': 'S',
    'Rondônia': 'N',
    'Ceará': 'NE',
    'Maranhão': 'NE',
    'Rio Grande do Sul': 'S',
    'Paraíba': 'NE',
    'Distrito Federal': 'CO',
    'Alagoas': 'NE',
    'Amapá':'N',
    'Piauí': 'NE',
    'Brasília': 'CO',
    'Tocantins': 'N',
    'Rio Grande do Norte': 'NE',
    'Sergipe': 'NE',
    'Minas Gerais/Goiás/Distrito Federal': 'CO',
    'Santa Catarina-Rio Grande do Sul': 'S'
}

# criando coluna com as regiões
teste2['regiao'] = teste2['estado_ou_provincia'].apply(lambda x: regioes[str(x)])

# coluna com estado, regiao
teste2['regiao_e_estado'] = teste2['estado_ou_provincia'] + ', ' + teste2['regiao']

# summing over regions to sort
sorting = teste2.groupby(['regiao_e_estado', 'regiao']).sum()['counts'].reset_index(
                                                                ).rename(columns={'counts':'soma'})

# sorting per most representative region and, then, by most representative state
regioes = sorting.sort_values('soma', ascending=False)['regiao'].unique()
df = pd.DataFrame()
for reg in regioes:
    temp = sorting[sorting['regiao'] == reg].sort_values('soma', ascending=False)
    df = pd.concat([df, temp])
    
sorting = df['regiao_e_estado'].unique()

# p.s.: this approach was not producing the desired outcome
# sorting = sorting.sort_values(['regiao','soma'], ascending=False)['regiao_e_estado'].unique()

In [16]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# filtering database (removing Brazil)
db = teste2

# auxiliar variables for encoding fields
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
y_labels = sorting
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 50))

g1 = alt.Chart(db, title='Total of collected specimens in Brazilian States').mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year', 
             scale= alt.Scale(domain= x_labels)),
    y= alt.Y('regiao_e_estado', type="nominal",title='State, Region',
             scale= alt.Scale(domain= y_labels),
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', title= 'Counts',
                  legend= alt.Legend(orient= 'bottom', direction= 'horizontal', tickCount= 5),
                  scale= alt.Scale(domain= counts, range=[30,130], zero=False)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Familia', 
                    legend= alt.Legend(columns=8, symbolLimit=50, orient='bottom', direction='horizontal'), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_brazilian_states.html')
# g1

<br>
<br>

<font color='red' size='5'> Separating this last chart per time periods </font>

In [24]:
# periods: <1938, >=1938 e <= 1985, > 1985 
ano_corte1 = 1938
ano_corte2 = 1985

periodo1 = teste2[teste2['ano_coleta'] < 1938].copy()
periodo2 = teste2[(teste2['ano_coleta'] >= 1938) & (teste2['ano_coleta'] <= 1985)].copy()
periodo3 = teste2[teste2['ano_coleta'] > 1985].copy()

### 1st period: before 1938

In [27]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# filtering database (removing Brazil)
db = periodo1

# auxiliar variables for encoding fields
# families = [f for f in cores_familia.keys() if f in db['familia'].unique()]
# cores_temp = [cores_familia[f] for f in families]
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
# y_labels = [l for l in sorting if l in db['regiao_e_estado'].unique()]
y_labels = sorting
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 20))

g1 = alt.Chart(db, title='Total of collected specimens in Brazilian States (before 1938)',
              width=500, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year',
            scale= alt.Scale(domain= x_labels)),
    y= alt.Y('regiao_e_estado', type="nominal",title='State, Region',
            scale= alt.Scale(domain= y_labels),
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Região'),
    size=alt.Size('counts', title= 'Counts',
                  legend= alt.Legend(orient= 'right', direction='horizontal', tickCount=4),
                  scale= alt.Scale(domain=counts, range=[30,130], zero=True)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), 
                                     range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_brazilian_states-before1938.html')
# g1

### 2nd period: from 1938 to 1985

In [21]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# filtering database (removing Brazil)
db = periodo2

# auxiliar variables for encoding fields
# families = [f for f in cores_familia.keys() if f in db['familia'].unique()]
# cores_temp = [cores_familia[f] for f in families]
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
y_labels = [l for l in sorting if l in db['regiao_e_estado'].unique()]
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 50))


g1 = alt.Chart(db, title='Total of collected specimens in Brazilian States (from 1938 to 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year',
            scale= alt.Scale(domain= x_labels)),
    y= alt.Y('regiao_e_estado', type="nominal",title='State, Region',
             scale= alt.Scale(domain= y_labels),
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Region'),
    size=alt.Size('counts', title= 'Counts',
                  legend= alt.Legend(orient= 'right', direction='horizontal', tickCount=4),
                  scale= alt.Scale(domain=counts, range=[30,130], zero=True)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), 
                                     range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_brazilian_states-from1938to1985.html')
# g1

### 3rd period: after 1985

In [23]:
# selector
select_family = alt.selection_multi(fields=['familia'], bind='legend')

# filtering database (removing Brazil)
db = periodo3

# auxiliar variables for encoding fields
# families = [f for f in cores_familia.keys() if f in db['familia'].unique()]
# cores_temp = [cores_familia[f] for f in families]
x_labels = db.sort_values('ano_coleta')['ano_coleta'].unique()
y_labels = [l for l in sorting if l in db['regiao_e_estado'].unique()]
counts = db['counts'].unique()
counts = list(range(min(counts), max(counts), 50))

g1 = alt.Chart(periodo3, title='Total of collected specimens in Brazilian States (after 1985)',
              width=800, height=600).mark_circle(opacity=0.8).encode(
    x= alt.X('ano_coleta', type="ordinal", title='Sampling Year',
            scale= alt.Scale(domain= x_labels)),
    y= alt.Y('regiao_e_estado', type="nominal",title='State, Region',
            sort= sorting, axis=alt.Axis(labels=True)), 
    y2 = alt.Y2('regiao', title='Region'),
    size=alt.Size('counts', title= 'Counts',
                  legend= alt.Legend(orient= 'right', direction='horizontal', tickCount=4),
                  scale= alt.Scale(domain=counts, range=[30,130], zero=True)), 
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip= alt.Tooltip(['regiao','estado_ou_provincia','ano_coleta','familia','counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/time-spacial/temporal_evolution_per_brazilian_states-after1985.html')
# g1

<br>

## Lat Long

In [18]:
def regiao(est):
    if est not in regioes.keys():
        return np.NAN
    else:
        return regioes[est]
    
NewTable['regiao'] = NewTable['estado_ou_provincia'].apply(regiao)

In [19]:
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

world = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1')

# world

In [20]:
# dynamic version
slider = alt.binding_range(min=1900, max=2016, step=1)
select_year = alt.selection_single(name=' ', fields=['ano_coleta'],
                                   bind=slider, init={'ano_coleta': 1935})

# data = NewTable[(NewTable['long'] < 10) & (NewTable['long'] >-150)].copy()
data = NewTable.copy()

back = alt.Chart(data).mark_circle(opacity=0.7, color='lightgray').encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem'])
)

front = alt.Chart(data).mark_circle().encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'ordem', 'familia'])
).add_selection(
    select_year
).transform_filter(
    select_year
)

equador = alt.Chart(data[data['lat'] == 0], width=800, height= 400).mark_rule(color='black').encode(
    y = alt.Y('lat:Q', axis=alt.Axis(grid=False))
)


g = alt.layer(world, back, front).properties(
    width= 800,
    height= 500
)  # tirei o equador

# ajustando tamanho de fonte
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/time-spacial/lat_long-worldwide_animated-year.html')
# g

<br>

<font color='red' size='5'>Lat long (layout 2 estático)</font>

In [21]:
# https://nextjournal.com/sdanisch/cartographic-visualization

# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

# database
db = NewTable.copy()
db['type_status'] = db['type_status'].astype(str)  # parsing into string to make selector work

tipos = db['type_status'].unique()  # lista temporária! ajustar depois!

# AJUSTAR CORES

from vega_datasets import data

front = alt.Chart(db).mark_point(filled=True).encode(
    longitude = alt.X('long', type='quantitative', title='Longitude'),
    latitude = alt.Y('lat', type='quantitative', title='Latitude'),
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos),
                      legend= alt.Legend(columns=3)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem','familia', 'type_status'])
).add_selection(select_order, select_type).transform_filter(select_order).transform_filter(select_type)

mapa = alt.layer(
     # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#f2f2f2'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(data.world_110m.url, 'countries')).mark_geoshape(
        fill='#white', stroke='black', strokeWidth=0.5
    ),
    front
).properties(
    width=800,
    height=500
)

mapa = mapa.project(
    type='naturalEarth1', scale=140
).configure_view(stroke=None)

mapa = mapa.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# mapa.save('./graphs/time-spacial/lat_long-worldwide-static-grid.html')
# mapa

In [22]:
# database
db = NewTable.copy()
db['type_status'] = db['type_status'].astype(str)  # parsing into string to make selector work

# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

teste = alt.Chart(db).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos), 
                     legend= alt.Legend(columns=3)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem', 'familia', 'type_status'])
).project(type='naturalEarth1').add_selection(select_order, 
                            select_type).transform_filter(select_order).transform_filter(select_type)

temp = (world + teste).properties(width=800, height=500).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-worldwide-static.html')
# temp

<br>

### Latin America

In [23]:
# seletores
select_order = alt.selection_multi(fields=['familia'], bind='legend')
select_type = alt.selection_multi(fields=['type_status'], bind='legend')

# database
db = NewTable.copy()
db['type_status'] = db['type_status'].astype(str)  # parsing into string to make selector work

tipos = db['type_status'].unique()  # lista temporária! ajustar depois!

# mapa
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

latinAmerica = alt.Chart(source).mark_geoshape(
    fill='white',
    stroke='gray'
).project('naturalEarth1', translate=[680, 150], scale=450)

# pontos
teste = alt.Chart(db).mark_point(filled=True).encode(
    longitude = alt.X('long:Q', title='Longitude'),
    latitude = alt.Y('lat:Q', title='Latitude'),
    color= alt.Color('familia:N', title='Family', 
                    legend= alt.Legend(columns=2, symbolLimit=50), 
                    scale= alt.Scale(domain= list(cores_familia.keys()), 
                                     range= list(cores_familia.values()))),
    shape = alt.Shape('type_status:N', title='Type', scale= alt.Scale(domain=tipos), 
                     legend= alt.Legend(columns=3)),
    tooltip = alt.Tooltip(['lat','long','pais','regiao','estado_ou_provincia',
                           'ano_coleta','mes_coleta', 'genero_atual','ordem', 'familia', 'type_status'])
).project(type='naturalEarth1', translate=[680, 150], scale=450).add_selection(select_order,
                                select_type).transform_filter(select_order).transform_filter(select_type)


# configurando grafico
temp = (latinAmerica + teste).properties(width=600, height=600).configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/time-spacial/lat_long-latinAmerica-static.html')
# temp

**That's it!**